# Setup

In [3]:
import numpy as np
from numpy import random
from scipy import stats
import csv
import pickle
import matplotlib
import matplotlib.pyplot as plt
from time import time
import pandas as pd

from gensim.models import Word2Vec

from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

%matplotlib inline
matplotlib.style.use('ggplot')

# Dataset

In [4]:
DATASET = 'amazon'

PATH_DATASET = '/Users/rafalencar/Documents/Datasets/Products_Catalog/'

In [5]:
dataset = Dataset('amazon_co-ecommerce_sample', PATH_DATASET)

NameError: name 'Dataset' is not defined

In [6]:
from pathlib import Path
filename = PATH_DATASET + 'amazon_co-ecommerce_sample.csv'
filenamejsonl = PATH_DATASET + 'amazon_co-ecommerce_dataset.jsonl'
columns_to_mantain = ['product_name', 'product_description', 'product_information',
                      'amazon_category_and_sub_category']

columns_new_name   = ['name', 'description', 'information' ,'category']

readFile = open(filename, 'r')
reader = csv.reader(readFile)
       
data = list()
for row in reader:
    data.append(row)
            
dataset = pd.DataFrame.from_dict(data)
dataset.columns = dataset.loc[0]
dataset = dataset.drop([0])
dataset = dataset.dropna()

dataset = dataset[columns_to_mantain]

dataset.head(5)

dataset.columns = columns_new_name
           
readFile.close()
        
dataset.to_json(filenamejsonl, orient='records', lines=True)

In [7]:
dataset = pd.read_csv(PATH_DATASET + 'amazon_co-ecommerce_sample_Dataset.csv')
dataset = dataset.replace(np.nan, '', regex=True)

dataToLabel = pd.read_csv(PATH_DATASET + 'amazon_co-ecommerce_sample_NoLabel.csv')
dataToLabel = dataToLabel.replace(np.nan, '', regex=True)

print('Not Labeled Dataset Shape: ', dataToLabel.shape[0])
print('Labeled Dataset Shape: ', dataset.shape[0])
dataset.head(5)

Not Labeled Dataset Shape:  690
Labeled Dataset Shape:  9310


,product_name,product_information,product_description,amazon_category_and_sub_category,amazon_category_and_sub_category.1
0,Hornby 2014 Catalogue,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,Hobbies > Model Trains & Railway Sets > Rail V...,Hobbies
1,FunkyBuys® Large Christmas Holiday Express Fes...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,Hobbies > Model Trains & Railway Sets > Rail V...,Hobbies
2,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Hobbies > Model Trains & Railway Sets > Rail V...,Hobbies
3,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Hobbies > Model Trains & Railway Sets > Rail V...,Hobbies
4,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Technical Details Item Weight159 g Product Dim...,Product Description Hornby RailRoad 0-4-0 Gild...,Hobbies > Model Trains & Railway Sets > Rail V...,Hobbies


In [ ]:
CATEGORY_FL = "amazon_category_and_sub_category.1"

labels = dataset[CATEGORY_FL].transform(lambda x: x.split(' > ')[0])
labels_freq = labels.value_counts() / labels.count()
labels_freq.plot(kind='bar', figsize=(15,4))
plt.show()

In [ ]:
CATEGORY    = "amazon_category_and_sub_category"

min_samples = 2
group_data = dataset[CATEGORY].value_counts()
group_data = group_data[group_data >= min_samples]
labels_freq = group_data / dataset[CATEGORY].count()

print('All labels with minimum %d samples: %d' % (min_samples, labels_freq.count()))
labels_freq.plot(kind='barh', figsize=(15, 80), width=0.5)
plt.show()

In [ ]:
MAX_SAMPLES = 1
groupby = dataset.groupby(CATEGORY).product_name.count()
filtered = groupby[groupby <= MAX_SAMPLES]

#Observação, filtrar os labels pelo número de exemplares
print("All Labels with maximum %d samples: " % MAX_SAMPLES, filtered.shape[0])

In [ ]:
datasetFilt = dataset[~dataset[CATEGORY].isin(filtered.index)]
labels = datasetFilt[CATEGORY].unique()
LABELS = labels.shape[0]
print("Total Filtered Labels: ", labels.shape[0])

In [ ]:
descr_size = []
X_used = "product_description"
for item in dataset[X_used]:
    split = item.split()
    descr_size.append(len(split))

mode = stats.mode(descr_size)
mean = np.mean(descr_size)
std  = np.std(descr_size)
maxL = max(descr_size)
minL = min(descr_size)

ratio = dataset.shape[0]/mean

print('Mode Description Length:', mode.mode[0], 'with', mode.count[0], 'samples') 
print('Mean Description Length: ', round(mean,2))  
print('Std Description Length: ', round(std,2))
print('Max Description Length: ', maxL)  
print('Min Description Length: ', minL) 
print('S/W Description Ratio: ', int(ratio)) 

plt.figure(figsize=(25,4))
plt.hist(descr_size, bins=maxL-1)
plt.show()

In [ ]:
info_size = []
X_used = "product_information"
for item in dataset[X_used]:
    split = item.split()
    info_size.append(len(split))
    
mode = stats.mode(info_size)
mean = np.mean(info_size)
std  = np.std(info_size)
maxL = max(info_size)
minL = min(info_size)

ratio = dataset.shape[0]/mean
  
print('Mode Description Length:', mode.mode[0], 'with', mode.count[0], 'samples')   
print('Mean Title Length: ', round(mean,2)) 
print('Std Title Length: ', round(std,2))  
print('Max Title Length: ', maxL)  
print('Min Title Length: ', minL)  
print('S/W Title Ratio: ', int(ratio))

plt.figure(figsize=(15,4))
plt.hist(info_size, bins=maxL-1)
plt.show()

In [ ]:
title_size = []
X_used = "product_name"
for item in dataset[X_used]:
    split = item.split()
    title_size.append(len(split))
    
mode = stats.mode(title_size)
mean = np.mean(title_size)
std  = np.std(title_size)
maxL = max(title_size)
minL = min(title_size)

ratio = dataset.shape[0]/mean
  
print('Mode Description Length:', mode.mode[0], 'with', mode.count[0], 'samples')   
print('Mean Title Length: ', round(mean,2)) 
print('Std Title Length: ', round(std,2))  
print('Max Title Length: ', maxL)  
print('Min Title Length: ', minL)  
print('S/W Title Ratio: ', int(ratio))

plt.figure(figsize=(15,4))
plt.hist(title_size, bins=maxL-1)
plt.show()

In [ ]:
def plot_frequency_distribution_of_ngrams(sample_texts, ngram_range=(1, 2), num_ngrams=50):
    # Create args required for vectorizing.
    kwargs = {
            'ngram_range': (1, 1),
            'dtype': 'int32',
            'strip_accents': 'unicode',
            'decode_error': 'replace',
            'analyzer': 'word',  # Split text into word tokens.
    }
    vectorizer = CountVectorizer(**kwargs)
    vectorized_texts = vectorizer.fit_transform(sample_texts)

    # This is the list of all n-grams in the index order from the vocabulary.
    all_ngrams = list(vectorizer.get_feature_names())
    num_ngrams = min(num_ngrams, len(all_ngrams))
    # ngrams = all_ngrams[:num_ngrams]

    # Add up the counts per n-gram ie. column-wise
    all_counts = vectorized_texts.sum(axis=0).tolist()[0]

    # Sort n-grams and counts by frequency and get top `num_ngrams` ngrams.
    all_counts, all_ngrams = zip(*[(c, n) for c, n in sorted(
        zip(all_counts, all_ngrams), reverse=True)])
    ngrams = list(all_ngrams)[:num_ngrams]
    counts = list(all_counts)[:num_ngrams]

    idx = np.arange(num_ngrams)
    plt.figure(figsize=(20,5))
    plt.bar(idx, counts, width=0.8, color='b')
    plt.xlabel('N-grams')
    plt.ylabel('Frequencies')
    plt.title('Frequency distribution of n-grams')
    plt.xticks(idx, ngrams, rotation=45)
    plt.show()
    
plot_frequency_distribution_of_ngrams(dataset[X_used], ngram_range=(1, 2), num_ngrams=50)

In [ ]:
labels = dataset[CATEGORY].unique()
LABELS = labels.shape[0]
print("Total Labels: ", labels.shape[0])

# Embedding

### Save Data Models

In [ ]:
X_used = 'product_information'
dataX = dataset[X_used]

if X_used == 'product_name': 
    MAX_SEQUENCE_LENGTH = 1.1*max(title_size) 
elif X_used == 'product_description':  
    MAX_SEQUENCE_LENGTH = 1.1*max(descr_size)
else:
    MAX_SEQUENCE_LENGTH = 1.1*max(info_size)

In [ ]:
EMBEDDING_DIM = 100

PATH_DATA_MODELS   = 'data_models/'

FILE_WORD2VEC      = PATH_DATA_MODELS + DATASET + '_' + X_used +'_word2vec_s' + str(EMBEDDING_DIM) + '.model'
FILE_TOKENIZER     = PATH_DATA_MODELS + DATASET + '_' + X_used +'_tokenizer.sav'
FILE_VECTORIZER    = PATH_DATA_MODELS + DATASET + '_' + X_used +'_vectorizer.sav'
FILE_LABEL_ENCODER = PATH_DATA_MODELS + DATASET +'_label_encoder.sav'

### Word2Vec

In [ ]:
sentences = []
for item in dataX:
    split = item.split()
    sentences.append(split)
    
start = time()
word2vec = Word2Vec(sentences, size=EMBEDDING_DIM)#, window=5, min_count=1, workers=4)
print("Execution Time:", round(time() - start, 2), 's')
print("Number of Word Vectors:", len(list(word2vec.wv.vocab)))

# Save Word2Vec
word2vec.save(FILE_WORD2VEC)

### Tokenizer

In [ ]:
MAX_NB_WORDS = 20000
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(dataX)
print("Number of Tokens:",len(tokenizer.word_counts))

# Save Tokenizer
pickle.dump(tokenizer, open(FILE_TOKENIZER, 'wb'))

### Vectorizer

In [ ]:
MAX_NB_WORDS = 20000
vectorizer  = TfidfVectorizer(max_features=MAX_NB_WORDS, stop_words='english')
vectorizer.fit(dataX)

# Save Vectorizer
pickle.dump(vectorizer, open(FILE_VECTORIZER, 'wb'))

### Label Encoder

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(labels)

# Save Label Encoder 
pickle.dump(le, open(FILE_LABEL_ENCODER, 'wb'))

# Classification Tree

In [ ]:
tree = Tree('Root')    
for label in labels:
    split = label.split(' > ')
    tree = tree.populateTree(split, 0)

In [ ]:
x_train = dataX
y_train = dataset[CATEGORY]
tree.sampleTree(x_train, y_train, 0)
print()